In [146]:
name = 'PTTEP'
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

In [147]:
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,PTTEP,3.0000,0.0000,4.2500,0.0000,7.2500,0,2023-02-10,2023-04-18,,0


In [148]:
url = 'https://classic.settrade.com/C04_07_stock_rightsandbenefit_p1.jsp?txtSymbol=' \
+ name + '&ssoPageId=15&selectPage=7'
# Set the encoding in the headers parameter
headers = {'encoding': 'utf-8'}
r = requests.get(url, headers=headers)
html = r.text
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')
df = pd.read_html(str(table), encoding='utf-8')[0]

In [149]:
# Specify the new col names in a list
new_col_names = ['col 1', 'col 2', 'col 3','col 4', 'col 5', 'col 6','col 7', 'col 8', 'col 9']
# Assign the new col names to the cols attribute of the DataFrame
df.columns = new_col_names
# Drop the second and third columns
df = df.drop(df.columns[[1, 2, 4, 6, 8]], axis=1)
df

,col 1,col 4,col 6,col 8
0,15/08/22,26/08/22,4.25,01/01/22-30/06/22
1,10/02/22,18/04/22,3.00,NaN
2,11/08/21,27/08/21,2.00,01/01/21-30/06/21
3,01/03/21,26/04/21,2.75,NaN
4,13/08/20,28/08/20,1.50,01/01/20-30/06/20
5,13/02/20,10/04/20,3.75,NaN
6,08/08/19,23/08/19,2.25,NaN
7,12/02/19,11/04/19,3.25,NaN
8,08/08/18,24/08/18,1.75,NaN
9,07/02/18,12/04/18,2.75,NaN


In [104]:
# Keep rows with position 0 and 2
mask = df.index.isin([1])
df_filtered = df[mask].copy()
df_filtered

,col 1,col 4,col 6,col 8
1,07/03/22,06/05/22,1.1,01/01/21-31/12/21


In [85]:
# Convert the 'date' column to datetime
df_filtered['col 1'] = pd.to_datetime(df_filtered['col 1'], format='%d/%m/%y')
df_filtered['col 4'] = pd.to_datetime(df_filtered['col 4'], format='%d/%m/%y')

# Format the 'date' column as strings using the strftime function
df_filtered['col 1'] = df_filtered['col 1'].dt.strftime('%Y-%m-%d')
df_filtered['col 4'] = df_filtered['col 4'].dt.strftime('%Y-%m-%d')

df_filtered['col 1'] = pd.to_datetime(df_filtered['col 1'])
df_filtered['col 4'] = pd.to_datetime(df_filtered['col 4'])
df_filtered['col 6'] = df_filtered['col 6'].astype('float')

In [86]:
# Define a lambda function that splits the date range into start and end dates, converts them to datetime and formats them as strings
convert_dates = lambda x: pd.to_datetime(x.split('-')[0], format='%d/%m/%y').strftime('%Y-%m-%d') + '-' + pd.to_datetime(x.split('-')[1], format='%d/%m/%y').strftime('%Y-%m-%d')

# Apply the lambda function to the 'date_range' column
df_filtered['col 8'] = df_filtered['col 8'].apply(convert_dates)

# Print the modified DataFrame
print(df_filtered)

       col 1      col 4  col 6                  col 8
1 2022-03-08 2022-05-13   0.24  2021-07-01-2021-12-31


In [87]:
xdate = df_filtered.iloc[0, 0]
# Add 1 year to the Timestamp
xdate = xdate.replace(year=xdate.year+1)
xdate = xdate.date()
# Print the modified Timestamp
print(xdate)

2023-03-08


In [88]:
pdate = df_filtered.iloc[0, 1]
# Add 1 year to the Timestamp
pdate = pdate.replace(year=pdate.year+1)
pdate = pdate.date()
# Print the modified Timestamp
print(pdate)

2023-05-13


In [89]:
q4 = df_filtered.iloc[0, 2]
print(q4)

0.24


In [72]:
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,PSH,0.5000,0.0000,0.3100,0.0000,0.8100,0,2023-03-10,2023-05-20,None,0


In [73]:
sqlUpd = """
UPDATE dividend
SET q4 = %s, dividend = q1+q2+q3+q4, xdate = '%s', paiddate = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (q4, xdate, pdate, name)
rp = const.execute(sqlUpd)

In [74]:
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,PSH,0.6500,0.0000,0.3100,0.0000,0.9600,0,2023-03-10,2023-05-20,None,0


### Delete dividend record that no longer interested

In [145]:
sqlDel = """
DELETE FROM dividend
WHERE name = 'SCB'"""
rp = const.execute(sqlDel)
rp.rowcount

1